In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,BloomTokenizerFast


checkpoint = "test_model"#"yuanzhoulvpi/chinese_bloom_7b_chat"#"bigscience/bloomz-3b" #"bigscience/bloom-7b1"#  "output_dir/checkpoint-8260"#

tokenizer = BloomTokenizerFast.from_pretrained("D:/dddd/bloom_560m")
model_new = AutoModelForCausalLM.from_pretrained("test_model").half().cuda()
model_raw = AutoModelForCausalLM.from_pretrained("D:/dddd/bloom_560m").half().cuda()

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

from typing import Optional
def generate_input(instruction:Optional[str]= None, input_str:Optional[str] = None) -> str:
    if input_str is None:
        return PROMPT_DICT['prompt_no_input'].format_map({'instruction':instruction})
    else:
        return PROMPT_DICT['prompt_input'].format_map({'instruction':instruction, 'input':input_str})


In [15]:
with open("map_index.json", mode='r', encoding='utf-8') as fin:
    import json
    oi2ni = json.loads(fin.read())
oi2ni = {int(k):v for k,v in oi2ni.items()}
oi2ni
text_inputs = "你是谁"
inputs = tokenizer.encode(text_inputs, return_tensors="pt")
inputs2 = tokenizer.encode(text_inputs, return_tensors="pt")
print(f"old: {inputs}")

import torch

def convertid(x):

    if x < min(oi2ni.keys()):
        return x 
    else:
        return oi2ni.get(x, 0)


inputs.apply_(lambda x: convertid(x))
print(f"new: {inputs}")


old: tensor([[21900, 15352]])
new: tensor([[8421, 6425]])


In [13]:
# def convertid_rev(x):
#     if x < min(oi2ni.item)

oi2ni_rev = {v:k for k, v in oi2ni.items()}

def convertid_rev(x):
    if x < min(oi2ni.keys()):
        return x 
    else:
        return oi2ni_rev.get(x,0)


test2 = torch.LongTensor([12051, 20318,  3997, 23533,  3665,2])
test2.apply_(convertid_rev)
test2


tensor([35187, 71479,  8433, 87152,  7561,     2])

In [3]:
model_new.forward(input_ids=inputs.to('cuda')).logits.shape

torch.Size([1, 5, 52922])

In [4]:
model_raw.forward(input_ids=inputs2.to('cuda'),return_dict=True).logits.shape

torch.Size([1, 5, 250880])

In [74]:
model_raw.forward(input_ids=inputs2.to('cuda'),return_dict=True)

odict_keys(['logits', 'past_key_values'])

In [5]:
model_raw.lm_head.weight[:10, :],model_new.lm_head.weight[:10, :]

(tensor([[-0.0098, -0.0048, -0.0109,  ..., -0.0407,  0.0098,  0.0213],
         [ 0.0040, -0.0131,  0.0141,  ..., -0.0431, -0.0002, -0.0118],
         [ 0.0078,  0.0246,  0.0038,  ..., -0.0433, -0.0064,  0.0135],
         ...,
         [ 0.0080,  0.0057, -0.0108,  ..., -0.0429, -0.0013, -0.0279],
         [ 0.0209, -0.0093,  0.0015,  ..., -0.0422, -0.0132,  0.0131],
         [ 0.0089, -0.0001,  0.0003,  ..., -0.0434, -0.0122, -0.0062]],
        device='cuda:0', dtype=torch.float16, grad_fn=<SliceBackward0>),
 tensor([[-0.0098, -0.0048, -0.0109,  ..., -0.0407,  0.0098,  0.0213],
         [ 0.0040, -0.0131,  0.0141,  ..., -0.0431, -0.0002, -0.0118],
         [ 0.0078,  0.0246,  0.0038,  ..., -0.0433, -0.0064,  0.0135],
         ...,
         [ 0.0080,  0.0057, -0.0108,  ..., -0.0429, -0.0013, -0.0279],
         [ 0.0209, -0.0093,  0.0015,  ..., -0.0422, -0.0132,  0.0131],
         [ 0.0089, -0.0001,  0.0003,  ..., -0.0434, -0.0122, -0.0062]],
        device='cuda:0', dtype=torch.float16,

In [80]:
model_row.lm_head.weight.shape,model.lm_head.weight.shape

(torch.Size([250880, 1024]), torch.Size([52922, 1024]))

In [17]:
for i in range(1):
    print("*"*80)
    outputs = model_new.generate(inputs.to('cuda'),num_beams=3,
                            max_new_tokens=512,
                            do_sample=False, 
                            top_k=10,
                            penalty_alpha=0.6,
                            temperature=0.8,
                            repetition_penalty=1.2)
    print(outputs[0])
    a = outputs[0].to('cpu')
    a.apply_(convertid_rev)
    print(tokenizer.decode(a))

# 12051, 20318,  3997, 23533,  3665

********************************************************************************
tensor([8421, 6425,    2], device='cuda:0')
你是谁</s>


In [16]:
for i in range(1):
    print("*"*80)
    outputs = model_raw.generate(inputs2.to('cuda'),num_beams=3,
                            max_new_tokens=512,
                            do_sample=False, 
                            top_k=10,
                            penalty_alpha=0.6,
                            temperature=0.8,
                            repetition_penalty=1.2)
    print(outputs[0])
    print(tokenizer.decode(outputs[0]))

********************************************************************************
tensor([21900, 15352,     2], device='cuda:0')
你是谁</s>


In [8]:
tokenizer.decode([2])

'</s>'

In [31]:
model.lm_head, model.lm_head.weight.shape

(Linear(in_features=1024, out_features=52922, bias=False),
 torch.Size([52922, 1024]))

In [33]:
from torch import nn

r1 = nn.Linear(4, 2, bias=False)
r1, r1.weight.shape , r1.weight

(Linear(in_features=4, out_features=2, bias=False),
 torch.Size([2, 4]),
 Parameter containing:
 tensor([[-0.0460,  0.2576, -0.1373,  0.0154],
         [ 0.2853, -0.2333,  0.4222, -0.2241]], requires_grad=True))

In [36]:
r2 = nn.Linear(4,2, bias=False)

r2.weight.data = r1.weight.data.clone()

r2, r2.weight.shape, r2.weight

(Linear(in_features=4, out_features=2, bias=False),
 torch.Size([2, 4]),
 Parameter containing:
 tensor([[-0.0460,  0.2576, -0.1373,  0.0154],
         [ 0.2853, -0.2333,  0.4222, -0.2241]], requires_grad=True))

In [38]:
import torch 

data1 = torch.randn(5,4)
data1.shape

torch.Size([5, 4])

In [40]:
torch.allclose(r1(data1), r2(data1))

True